### Object Detection 
Object detection has been a core computer vision problem from the begining. The idea in object detection is that we again start with an input image and we have a **fixed** set of categories. We need to detect these categories from the image. The idea is every time the categories appear in the input image we want to draw a box around it and predict the label. This is challenging task because we don't know how many object will appear in the input image.

In object localization the number of object is fixed but here there may be different number of objects therefore we can address object detection with single object as the localization.



![alt text](images/od1.PNG)


In single object detection problem we only need to output 4 numbers(co-ordinates) of the object. But in multiple object detection we need to output 4 co-ordinates for each object. For example, if an image contain 3 objects say three dogs or whatever then we will have to output 12 numbers.

![alt text](images/od3.PNG)

### Approaches

##### **Approach-1: Sliding window**
 In this approach we will take an images and break it down into many patches and feed them into convolutional neural network. The network will output the class of each patch. In addition to the label we also want another label(background) from the model for each patch. This will confirm us that there is no object in a certain patch for a corresponding position in the image.

 **How do we choose these crops?**
 This has been a very challenging problem. Because,
 - there could be any number of objects in the image. 
 - The object could be in any position, of any size, and of any spatial size.

**Brute Force sliding**: If we apply brute force sliding window approach then we could end up with many many thousand crops. Then we need to feed each of them in the gigantic Neural networks which would be computationally intractable.

![alt text](images/od2.PNG)
   
**Region Proposals**: Instead of brute force sliding, we do have a traditional computer vision techniques named **region proposal network** which uses traditional signal/image processing techniques to list some proposals. The region proposal network will look into the blobby regions in the input image then give us some set of candidate proposal regions where objects might be potentially found. These methods are fast.
One region proposal method is **selective search** which give us 2 thousand proposals where the objects might be found. So, most of them won't be object rather nosies. But the precision of getting object is high.

![alt text](images/od4.PNG)



### Training
**R-CNN**

Rather than applying the classification network to every possible of the image, we can first apply this region proposal network to the image. Then we can apply the classification model only to these proposed regions. This will be then computationally tractable. This idea was introduced in a paper **R-CNN**. The proposals are sometimes called "region of interests".

One problem with this method is that the regions could be of any size in the input image but our convolutional network need same size of inputs due to fully connected layers. Therefore we will take each of the proposals and wrap them into a fixed size to be suited for the downstream network. Then we will run these wrapped proposals through CNN which will classify each of the proposed regions. In addition to that the R-CNN will also predict a regression to correct the bounding box. Because the regions might not be perfetc for an object in the input image. Therfore the network will predict four numbers that are kind of offset or a correction to the box that was predicted at the region proposal state. Then again this is a multitask loss in this case.

**Q** Are the region proposals learned? 

No. Because the region proposal network is a fixed function.

**Porblem**: Again this method is computationally expensive. Because we need to fed those 2k proposals into the Convolutional network. Moreover, the proposal regionss are fixed, we are not learning them. This is also a problem.


![alt text](images/rcnn.PNG)

**Fast R-CNN**

In fast R-CNN we first feed the input image through a CNN and get a high resolution feature map. Now rather than cropping iamge, based on the region proposals directly we projet the region proposals onto the feature map then taking the crops. This allow us to reuse a lot of expensive convolutional computation across the entire image when we have many many crops per image.

Then again we reshape the crops into same shape for running thorugh downstream network. We do this with the help of **RoI** pooling layer. Then we can run these reshaped crops through some fully connected layer to get class scores and regression offsets to the bounding boxes. When we train this, we again have a multitask loss that trades off between these two constraints and during back propagation we can back prop through this entire thing jointly.


![alt text](images/fast-rcnn.PNG)

**RoI pooling**: It is kind of Max Pooling layer. Here we won't discuss about much. For now I am just adding some images of steps of RoI...

![alt text](images/od5.PNG) ![alt text](images/od6.PNG)
![alt text](images/od7.PNG) ![alt text](images/od8.PNG)
![alt text](images/od9.PNG)

Fast R-CNN is about 10 times faster than R-CNN. The computation time of all the regions using selective search is about 2 seconds. Now since we train these regions in a shared way by sharing convolutions across entire image that can process all these region proposals in less than a second together. So fast R-CNN is bottlenecked by the computing of these region proposals. **Faster R-CNN** solve this problem.

**Faster R-CNN**
In fast R-CNN, the problem was that we had fixed region proposals. In faster R-CNN we will make the network to predict these regions itself. Again we will take an input image, pass through a CNN and get a feature map. Now we will use a separate region proposal network which will work on top of the feature map and predicts its region proposals. The rest of the process is same as fast R-CNN.

In faster R-CNN the network performs four task at once. Region proposals need to predict if there is an object or not and the bounding box co-ordinates for each of the proposals. Moreover the final network at the end needs to do these two things again. The final class scores and a second round of bounding box regression to again correcting errors that may have come from regioned proposal state.




![alt text](images/od10.PNG) ![alt text](images/od11.PNG)

**Q**: How do make the region proposal network learn when we don't have ground truth region proposals for the region proposal network?

**Ans**: It's little bit hairy. We won't talk about here now. But the idea is that at any time we have a regioned proposal which has more than some threshold of overlap with the ground truth objects then we consider that region as positive and predict that region as a proposal. When we have less overlap we consider that region as negative don't mark as a region.

**Q**: Whata is the classification loss at the region proposal network?

**Ans**: Again, this self learning region proposal network is a complex one. We might talk about later. But it's making binary classification decision on some set of regions it's condidering. If the region is an object or not. Therefore we will use binary classification loss.

So far we have discussed different types of R-CNN. Here "R" stands for region. That means all the methods are region based obeject detection method. There are other methods available for object detection. Such as,
- YOLO(You Only Look Once)
- SSD(Single Shot Detection)

#### YOLO/SSD/RetinaNet

The idea is that rather than doing independent processing for each of these potential regions, instead we want to try and treat this like a regression problem and just make all these predictions all at once with some big convolutional network. 

**Procedure**: We take our input image and divide it into some course of grid(in our example it's 7x7). Then for each of the grid we imagine some set of the base bounding boxes. Here we have taken 3(a tall, a wide, and a square) but in practice we will take more than 3.

Now we want to predict an offset for each of the base bounding boxes to predict the location of the object of this base bounding box and a classification score for each of the boxes. So we end up predicting a giant tensors of 7x7x(5B+C). We have B based bounding boxes and each has 5 numbers, the offset(co-ordinates:4 numbers) and the classification score for that box and finally C classification scores for C categories. So the problem is now, we have an input image and we get a 3 dimensional output by using a giant CNN.


![alt text](images/od12.PNG)

The way the region proposal network works in faster R-CNN is quite similar of this YOLO method. There were some base bounding boxes over some grid of image and region proposal network does some classification and regression.

Object detection has lots of variables. Moreover, there can be different base networks. There is also hybrid method called **RFC** where the techniques from region-based detection and YOLO based detection are blended.

Therer are different hyperparameters e.g. what is the image size, how many region proposals do we need etc.. New works have done on object detection like **CPR**.

![alt text](images/od13.PNG)


Once we have gainde these ideas, we can apply more than one techniqes in particular problems. For example we can apply object detection and captioning together in a same problem. Where now we want to classify each of the bounding box as well as a caption corresponding to the object in that bounding box.